# HDR gathering
Like basic gather, but with multiple pictures taken at different attenuations.

## Setup
Run `python f_camera_photonics\tcp_link.py` on the lab computer. Only one thing can use the analog port at a time, so make sure it is grabbing from the digital only.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import cv2
import sys, os
import numpy as np
import pickle
import matplotlib.pyplot as plt

sys.path.append('..')
# import f_camera_photonics
from f_camera_photonics.component_capture import single_shot
from f_camera_photonics import cvshow
from f_camera_photonics.peak_finder import f_camera_photonics, main, get_all_config, PortArray
from f_camera_photonics.tcp_link import remote_call, capture, attenuate, unpack_image

In [ ]:
remote = True
datadir = '../measurement_data/sheep02-hidras'

sample_name = 'north-maximumTap-B'  # change this every device
def to_file(extension):
    return os.path.join(datadir, sample_name + extension)

In [ ]:
def save_matrix(HDR, Powers):
    pickle.dump(Powers, open(to_file('.pickle'), 'wb'))
    
    metadata = dict(HDR_vals=list(HDR),
                    laser_Amps=0.130)  # this second row you fill out manually
    pickle.dump(metadata, open(to_file('.metadata.pickle'), 'wb'))
    
    fi, ax = plt.subplots(figsize=(10, 5))
    ax.loglog(10 ** (-HDR / 10), Powers, '.-')
    ax.grid(which='minor')
    plt.xlabel('Attenuation')
    plt.ylabel('Port power')
    plt.title(sample_name)
    plt.savefig(to_file('.png'))

## Preview valid box
You can do this with the lamp on, but make sure to turn it off afterward.

You will be given a preview of the valid box

In [ ]:
if remote:
    remote_call(attenuate, atten=0)
    img = unpack_image(remote_call(capture, avgcnt=2))
else:
    atten_db(atten=10)
    img = single_shot()
    
cfg=get_all_config()
if cfg.valid_box is not None:
    x, y, w, h = cfg.valid_box
#     cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 2)
    
cv2.imwrite(to_file('.tif'), img)
# cv2.imwrite(os.path.join(datadir, 'typical-00dB-bright.tif'), img)  # various pictures you want to save otherwise

# cvshow(img)

### If you are satisfied with the valid box, turn out the lamp and move on

## Find the port box spec on the saved image
You will be given a preview of the port boxes

In [ ]:
# No file artifacts
# pout = f_camera_photonics(filename=tif_file, box_spec=None, max_box_width=10)
# port_arr = PortArray.from_dict(pout)

if True:  # Use the manual box spec or not
    # This is set up so you put the center spot right next to the line of dead pixels
    all_x = 390
    ylist = np.linspace(0, 77, 5) + 386
    my_boxspec = [[all_x, y, 6] for y in ylist]
    ref_port = [45, 520, 6]
    my_boxspec = [ref_port] + my_boxspec
else:
    my_boxspec = None


# File artifacts
main(filename=to_file('.tif'), box_spec=my_boxspec, max_box_width=10)
with open(to_file('.json'), 'r') as fx:
    port_arr = PortArray.from_dict(json.load(fx))
port_arr.sort_by('y')
    
print(port_arr)

### If you are satisfied with port boxes, move on

## HDR image taking

In [ ]:
HDR_vals = np.linspace(0, 30, 30)
P_matrix = np.zeros((len(HDR_vals), len(port_arr)))
for i, db in enumerate(HDR_vals):
    if remote:
        remote_call(attenuate, atten=db)
        img = unpack_image(remote_call(capture, avgcnt=30))
    else:
        atten_db(db)
        img = single_shot()
    print('i =', i)
    P_matrix[i, :] = port_arr.calc_powers(img)
remote_call(attenuate, atten=10)  # reset

In [ ]:
# if you like it save it
save_matrix(HDR_vals, P_matrix)

In [ ]:
plt.loglog(10 ** (-HDR_vals / 10), P_matrix[:, 1:]-np.min(P_matrix)+1e2, '.-')